# Fine-tuning API 활용 예시

원본 파일: `llm-modeling-lab.jsonl`

```json
{
  "input": "아인슈페너 레귤러 사이즈로 부탁드리고, 콩국수 한 그릇 주세요.",
  "output": "- 분석 결과 0: 음식명:아인슈페너,옵션:레귤러,수량:1\n- 분석 결과 1: 음식명:콩국수,수량:한 그릇"
}
```

In [36]:
pip install jsonlines

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
import jsonlines

# 파일 준비

In [38]:
system_message = "너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다."

In [39]:
# 파일 경로 설정
input_filepath = '../local_data/llm-modeling-lab.jsonl'  # 원본 JSONL 파일 경로
output_filepath = '../local_data/order-understanding-data.jsonl'  # 변환된 JSONL 파일 저장 경로

# 시스템 메시지
system_message = "너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다."

# 변환을 위한 함수 정의
def convert_to_finetuning_format(input_filepath, output_filepath, system_message):
    # 입력 파일 읽기 및 출력 파일 쓰기
    with jsonlines.open(input_filepath, 'r') as infile, jsonlines.open(output_filepath, 'w') as outfile:
        for data in infile:
            # input과 output을 Fine-tuning API 포맷에 맞게 변환
            formatted_data = {
                "messages": [
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": data['input']},
                    {"role": "assistant", "content": data['output']}
                ]
            }
            # 변환된 데이터를 출력 파일에 작성
            outfile.write(formatted_data)

# 변환 함수 실행
convert_to_finetuning_format(input_filepath, output_filepath, system_message)

print(f"데이터가 '{output_filepath}' 파일에 성공적으로 저장되었습니다.")


데이터가 '../local_data/order-understanding-data.jsonl' 파일에 성공적으로 저장되었습니다.


데이터 분할
- 앞쪽 2,000개: train 
- 뒷쪽 1,000개: validation

**TODO**
- 200개 / 100개 조합으로 갯수를 줄여서 해보자

In [40]:
""" #윈도우 지원x
!head -n 200 ../local_data/order-understanding-data.jsonl > ../local_data/order-understanding-train.jsonl
!tail -n 100 ../local_data/order-understanding-data.jsonl > ../local_data/order-understanding-validation.jsonl
"""

' #윈도우 지원x\n!head -n 200 ../local_data/order-understanding-data.jsonl > ../local_data/order-understanding-train.jsonl\n!tail -n 100 ../local_data/order-understanding-data.jsonl > ../local_data/order-understanding-validation.jsonl\n'

In [41]:
# 파일 읽기
with open(output_filepath, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# 상위 200줄을 train 파일로, 하위 100줄을 validation 파일로 저장
with open("../local_data/order-understanding-train.jsonl", 'w', encoding='utf-8') as f:
    f.writelines(lines[:200])

with open("../local_data/order-understanding-validation.jsonl", 'w', encoding='utf-8') as f:
    f.writelines(lines[-100:])

# Client 설정

In [42]:
from openai import OpenAI
import openai
import os
with open("../specs/.openai_api_key", "r") as f:
    ss = f.readline().strip()

os.environ["OPENAI_API_KEY"] = ss
client = OpenAI()

# 파일 업로드

In [43]:
training_file_name = "../local_data/order-understanding-train.jsonl"
validation_file_name = "../local_data/order-understanding-validation.jsonl"

In [44]:
def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id


training_file_id = upload_file(training_file_name, "fine-tune")
validation_file_id = upload_file(validation_file_name, "fine-tune")

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-mLpMfp1EXJLU5RR1SDn4ymN4
Validation file ID: file-nj3pRbqZgAWezHKH2JKXgKxZ


# 모델 미세 튜닝 job 생성

In [50]:
MODEL = "gpt-4o-mini-2024-07-18"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=MODEL,
    suffix="order-understanding",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-pJ9iNJuVSJb5yS1DUOCL1ilR
Status: validating_files


# 진행 상태 확인

In [46]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-0vvReXUu61rBn07YSH6rqk63
Status: validating_files
Trained Tokens: None


In [47]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Created fine-tuning job: ftjob-0vvReXUu61rBn07YSH6rqk63
Validating training file: file-mLpMfp1EXJLU5RR1SDn4ymN4 and validation file: file-nj3pRbqZgAWezHKH2JKXgKxZ


In [51]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None:
    raise RuntimeError(
        "Fine-tuned model ID not found. Your job has likely not been completed yet."
    )

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-4o-mini-2024-07-18:personal:order-understanding:ANcUYUPE


# 결과 모델 활용

In [52]:
completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[{"role":"system", "content":system_message},{"role": "user", "content": "아이스 초코라떼 두 잔 주세요. 그리고 함께 먹을 수 있는 빵 하나도 주세요."}]
)
print(completion.choices[0].message)


ChatCompletionMessage(content='- 분석 결과 0: 음식명:아이스 초코라떼,수량:두 잔\n- 분석 결과 1: 음식명:빵,수량:하나', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)
